This part is an implemention with Python of query 1 and query 6, which can query the same data as q1.sql and q6.sql from the DBMS. I will read the data directly from tbl files, and use data frame to filter the conditions.

1. implemention of q6.sql and q1.sql with SF-1 data.

In [1]:
# load data
import pandas as pd

# Define the column names based on the schema
columns = [
    'l_orderkey', 'l_partkey', 'l_suppkey', 'l_linenumber', 
    'l_quantity', 'l_extendedprice', 'l_discount', 'l_tax', 
    'l_returnflag', 'l_linestatus', 'l_shipdate', 'l_commitdate', 
    'l_receiptdate', 'l_shipinstruct', 'l_shipmode', 'l_comment'
]

df = pd.read_csv('/Users/jay/Desktop/lessons material/2024-2/Advanced data management/assignment 01/TPC-H_V3.0.1++/dbgen/SF-1/data/lineitem.tbl', 
                 sep='|', header=None, names=columns, index_col=False, engine='python')

print(df.head())



   l_orderkey  l_partkey  l_suppkey  l_linenumber  l_quantity  \
0           1     155190       7706             1          17   
1           1      67310       7311             2          36   
2           1      63700       3701             3           8   
3           1       2132       4633             4          28   
4           1      24027       1534             5          24   

   l_extendedprice  l_discount  l_tax l_returnflag l_linestatus  l_shipdate  \
0         21168.23        0.04   0.02            N            O  1996-03-13   
1         45983.16        0.09   0.06            N            O  1996-04-12   
2         13309.60        0.10   0.02            N            O  1996-01-29   
3         28955.64        0.09   0.06            N            O  1996-04-21   
4         22824.48        0.10   0.04            N            O  1996-03-30   

  l_commitdate l_receiptdate     l_shipinstruct l_shipmode  \
0   1996-02-12    1996-03-22  DELIVER IN PERSON      TRUCK   
1   1996-0

In [93]:
# implement the query 06
import time
import numpy as np

quantity_list = np.arange(1,24,1)
start_time = time.time()*1000

# Filter the DataFrame based on conditions
filtered_df = df[
    (df['l_discount'].between(0.05, 0.07)) &
    (df['l_quantity'] < 24) &
    (df['l_shipdate'] >= '1994-01-01') &
    (df['l_shipdate'] < '1995-01-01')
]

# Calculate the revenue
filtered_df['revenue'] = filtered_df['l_extendedprice'] * filtered_df['l_discount']

# Sum the revenue
total_revenue = filtered_df['revenue'].sum()

# End timing the query
end_time = time.time()*1000

# Calculate the query time
query_time = end_time - start_time

print(f"Query execution time: {query_time:.4f} ms")
print(total_revenue)

Query execution time: 64.5300 ms
123141078.22829999


/var/folders/z7/xnwcgx9d04q493pxbdg5xkzh0000gn/T/ipykernel_4703/3800549251.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['revenue'] = filtered_df['l_extendedprice'] * filtered_df['l_discount']


In [59]:
# implement the query 01

df['l_shipdate'] = pd.to_datetime(df['l_shipdate'], format='%Y-%m-%d')
df['l_commitdate'] = pd.to_datetime(df['l_commitdate'], format='%Y-%m-%d')
df['l_receiptdate'] = pd.to_datetime(df['l_receiptdate'], format='%Y-%m-%d')

# Start query
start_time = time.time()*1000

# Filter DataFrame for l_shipdate less than or equal to 90 days before '1998-12-01'
cutoff_date = pd.to_datetime('1998-12-01') - pd.DateOffset(days=90)
filtered_df = df[df['l_shipdate'] <= cutoff_date]

# Group by l_returnflag and l_linestatus, then aggregate
result = filtered_df.groupby(['l_returnflag', 'l_linestatus']).agg(
    sum_qty=('l_quantity', 'sum'),
    sum_base_price=('l_extendedprice', 'sum'),
    sum_disc_price=('l_extendedprice', lambda x: (x * (1 - filtered_df.loc[x.index, 'l_discount'])).sum()),
    sum_charge=('l_extendedprice', lambda x: (x * (1 - filtered_df.loc[x.index, 'l_discount']) * (1 + filtered_df.loc[x.index, 'l_tax'])).sum()),
    avg_qty=('l_quantity', 'mean'),
    avg_price=('l_extendedprice', 'mean'),
    avg_disc=('l_discount', 'mean'),
    count_order=('l_orderkey', 'count')
).reset_index()

# Sort the results by l_returnflag and l_linestatus
result_q1 = result.sort_values(by=['l_returnflag', 'l_linestatus'])

# end query
end_time = time.time()*1000

query_time = end_time - start_time
print(f"Query execution time: {query_time:.4f} ms")

Query execution time: 1726.8889 ms


In [100]:
# verify the result of query 01

# read the standard value from csv file
correct_results = pd.read_csv("/Users/jay/Desktop/lessons material/2024-2/Advanced data management/assignment 01/TPC-H_V3.0.1++/dbgen/SF-1/results/q01.res.csv")
# transfer all of the number to .2f
def format_number(value):
    if isinstance(value, (int, float)):
        return f"{value:.2f}"
    return value 
format_res_q1 = result_q1.applymap(format_number)
format_correct = correct_results.applymap(format_number)

comparison_result = format_res_q1 == format_correct
if comparison_result.values.all():
    print("Query 1 0 unacceptable missmatches.")
else:
    print("ERROR: Query 1 has missmatches!")

Query 1 0 unacceptable missmatches.


/var/folders/z7/xnwcgx9d04q493pxbdg5xkzh0000gn/T/ipykernel_4703/3842612555.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  format_res_q1 = result_q1.applymap(format_number)
/var/folders/z7/xnwcgx9d04q493pxbdg5xkzh0000gn/T/ipykernel_4703/3842612555.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  format_correct = correct_results.applymap(format_number)
